<a href="https://colab.research.google.com/github/leonardo3108/IA368dd/blob/main/exercicios/Aula_5%20/Aula_5_Treino_Modelo_de_Linguagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Leonardo Augusto da Silva Pacheco'
print(f'Meu nome é {nome}')

Meu nome é Leonardo Augusto da Silva Pacheco


# Introdução

## Enunciado

Treinar um modelo de linguagem em dados em portugues
- Avaliar o modelo usando a perplexidade, que é simplesmente a exponencial de todas as losses do dataset de validação
- Iremos treinar o modelo para prever o próximo token dado os anteriores (também conhecido como Causal Language Modeling). Não confundir com o Masked Language Modeling (MLM), que consiste em prever tokens mascarados em uma dada sequência (ex: BERT's MLM)

Dicas:
- Usar como ponto de partida o modelo [OPT-125M](https://huggingface.co/facebook/opt-125m), que já foi treinado em 300B de tokens (maioria em Inglês)
- Usar este dataset reduzido do mc4 portugues, com ~300M de tokens: [mc4-pt-sample-1g.txt](https://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt)


## Abordagem

Segue uma descrição dos passos utilizados e documentação dos principais conceitos:

1. Baixamos o conjunto de dados mc4-pt-sample-1g.txt e preparamos os dados para treinamento, em batchs contendo sequências de número fixo de tokens.
   - preparação do dataset: https://huggingface.co/docs/datasets/loading
   - tokens em batch: https://huggingface.co/docs/datasets/about_map_batch
2. Baixamos o modelo pré-treinado OPT-125M e efetuamos o fine-tuning usando o dataset e por meio de modelagem de dados causal.
  - treinamento: https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/trainer
  - Causal language modeling with GPT: https://heartbeat.comet.ml/causal-language-modeling-with-gpt-d92c9cfe2d2a
4. Avaliamos o modelo treinado por meio da métrica perplexidade, e testamos a geração de texto (greedy decoder).
  - Perplexidade: https://huggingface.co/docs/transformers/main/en/perplexity
  - Greedy decoding: https://medium.com/geekculture/greedy-search-decoding-for-text-generation-62e6dad889b

Fontes de inspiração:
- slide da aula com o código gerado pelo ChatGPT
- notebook de exemplo da huggingface: [How to fine-tune a model on language modeling](https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb)
- notebook da [Mirelle](https://colab.research.google.com/drive/1WlSg5THKtALqYPDmqilfhjGkGHSYBJv7?usp=sharing#scrollTo=30Vu-BDcWrAa)



# Setup

## Preparação de pastas

In [1]:
!mkdir 'model_output'
!mkdir 'model_save'

## Hiperparâmetros

In [2]:
model_name = 'facebook/opt-125m'
max_seq_length=256
batch_size=16
epochs=3
model_output_dir='model_output'
model_save_dir='model_save'

## Instalação de libs

In [3]:
!pip install transformers
!pip install datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 38.2 MB/s eta 0:00:

## Importação de libs

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import math

## Utilização de GPUs para treinamento

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


In [6]:
if dev != 'cpu':
    !nvidia-smi

Mon Mar 27 10:31:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preparação dos dados

## Obtenção

In [7]:
 !wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2023-03-27 10:31:41--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 142.250.153.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G  33.0MB/s    in 38s     

2023-03-27 10:32:20 (31.3 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



## Criação do Dataset

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

dataset = load_dataset("text", data_files='sample-1gb.txt')
tokenized_dataset = dataset.map(lambda x : tokenizer(x['text'], truncation=True, padding="max_length", max_length=max_seq_length),
                                batched=True,
                                num_proc=4,
                                remove_columns=['text'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-6e39943746455e53/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map (num_proc=4):   0%|          | 0/250000 [00:00<?, ? examples/s]

## Teste do dataset

In [9]:
print(tokenized_dataset)
print(f"First seq: {len(tokenized_dataset['train']['input_ids'][0])} tokens - {tokenized_dataset['train']['input_ids'][0]}")

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 250000
    })
})
First seq: 256 tokens - [2, 17860, 4901, 289, 1526, 1076, 39428, 41, 366, 6, 1021, 295, 5874, 2089, 139, 263, 11619, 24505, 364, 7458, 3381, 281, 1192, 2849, 6009, 10, 139, 181, 5874, 39031, 4842, 3840, 1198, 6276, 3567, 9131, 7375, 1192, 1021, 263, 9379, 2359, 4, 7300, 2628, 102, 24988, 636, 808, 1829, 6, 14848, 1469, 424, 109, 524, 368, 263, 45345, 6, 2955, 2575, 3952, 3381, 4214, 364, 385, 1469, 424, 1296, 991, 879, 5410, 18769, 10, 542, 3381, 4214, 109, 11631, 30710, 139, 8550, 139, 4, 5082, 2359, 6, 181, 1322, 1755, 1192, 10, 29959, 3851, 109, 22, 5234, 2716, 368, 9, 23092, 113, 11, 1452, 9060, 475, 6439, 366, 263, 14848, 1322, 119, 3137, 1021, 687, 20329, 10, 4677, 1469, 763, 263, 45345, 4, 221, 1322, 1755, 1192, 1368, 1526, 7252, 22936, 1020, 263, 14848, 271, 2269, 5285, 263, 7252, 181, 5874, 428, 5895, 139, 1192, 6, 21045, 5511, 242, 6, 7935, 475, 5655, 6979,

# Treinamento

In [10]:
training_args = TrainingArguments(
    output_dir = model_output_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
   model = model,
   args = training_args,
   train_dataset = tokenized_dataset['train'],
   eval_dataset = tokenized_dataset['train'],
   data_collator = data_collator
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.677900,2.391012
2,2.409100,2.269489
3,2.323200,2.229376


TrainOutput(global_step=46875, training_loss=2.470055, metrics={'train_runtime': 22269.9463, 'train_samples_per_second': 33.678, 'train_steps_per_second': 2.105, 'total_flos': 9.7984512e+16, 'train_loss': 2.470055, 'epoch': 3.0})

In [11]:
model.save_pretrained(model_save_dir)
tokenizer.save_pretrained(model_save_dir)

('model_save/tokenizer_config.json',
 'model_save/special_tokens_map.json',
 'model_save/vocab.json',
 'model_save/merges.txt',
 'model_save/added_tokens.json',
 'model_save/tokenizer.json')

# Avaliação

## Cálculo da Perplexidade

In [12]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 9.29


## Teste da geração de texto

In [41]:
prompt = 'O café é uma bebida'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenizer(text=prompt)['input_ids']
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos tokens como entrada para o modelo.
    output = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = output['logits'][:, -1, :]  # Usamos apenas o ultimo token da sequencia
    predicted_id = torch.argmax(logits).item()  # extraindo o token de maior probabilidade (greedy decoding)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt.replace('</s>', ''))

O café é uma bebida que
O café é uma bebida que p
O café é uma bebida que pode
O café é uma bebida que pode ser
O café é uma bebida que pode ser consum
O café é uma bebida que pode ser consumida
O café é uma bebida que pode ser consumida em
O café é uma bebida que pode ser consumida em qual
O café é uma bebida que pode ser consumida em qualquer
O café é uma bebida que pode ser consumida em qualquer l
O café é uma bebida que pode ser consumida em qualquer lugar
O café é uma bebida que pode ser consumida em qualquer lugar,
O café é uma bebida que pode ser consumida em qualquer lugar, mas
O café é uma bebida que pode ser consumida em qualquer lugar, mas n
O café é uma bebida que pode ser consumida em qualquer lugar, mas não
O café é uma bebida que pode ser consumida em qualquer lugar, mas não é
O café é uma bebida que pode ser consumida em qualquer lugar, mas não é o
O café é uma bebida que pode ser consumida em qualquer lugar, mas não é o cas
O café é uma bebida que pode ser consumida em

In [ ]:
!zip model model_save/*